<a href="https://colab.research.google.com/github/GioSak27/Ejercicios_diplomado_2024/blob/main/proyecto_giovana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analisis operaciones en comercios electrónicos que utilizan alguna pasarela de pagos.

In [1]:
#Importamos librerias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.io as pio
import geopandas as gpd
import plotly.graph_objects as go
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
#fijamos ruta donde vive la data
%cd /content/drive/ My \Drive/Colab Notebooks/Archivos_excel
df = pd.read_excel('Trx_feb_aceptacion.xlsx')

/content/drive/ My Drive/Colab Notebooks/Archivos_excel


In [3]:
df.head()

,STATUS,RESPUESTA,DIA,MES,BANCO,Tipo_Banco,TIPO_TARJETA,PROCESAMIENTO,COMERCIO,MERCHANTID,CREACION_COMERCIO,TRX,MONTO,Con_Data_3D,CC_Inicio_TRX,CC_Inicio_Monto,Estado_Comercio,Origen_trx,Detalle_fallida
0,CANCELLED,"Due date expired, transaction cancelled",11,3,NaN,NaN,NaN,Agregador,ALGER WORLD TRAVEL SA DE CV,m6gcpxvaxs9fk40icugu,2022-03-01 17:58:06,1,36460.56,,0,0,ACTIVE,TPV,CANCELLED
1,CANCELLED,"Due date expired, transaction cancelled",11,3,NaN,NaN,NaN,Agregador,ALGER WORLD TRAVEL SA DE CV,m6gcpxvaxs9fk40icugu,2022-03-01 17:58:06,2,52412.06,3D,0,0,ACTIVE,TPV,CANCELLED
2,CANCELLED,"Due date expired, transaction cancelled",11,3,NaN,NaN,NaN,Agregador,ALTERNATIVAS BIOMEDICAS Y REGENERATIVAS SA DE CV,mckowzhogcrbl9fsbfjj,2022-03-29 18:06:38,2,28890.00,,0,0,ACTIVE,TPV,CANCELLED
3,CANCELLED,"Due date expired, transaction cancelled",11,3,NaN,NaN,NaN,Agregador,BIOPLAGA SOLUCIONES SUSTENTABLES SA DE CV,mbaaz3tzuclkfmkqd0li,2024-03-11 09:16:51,1,50000.00,,0,0,ACTIVE,TPV,CANCELLED
4,CANCELLED,"Due date expired, transaction cancelled",11,3,NaN,NaN,NaN,Agregador,CANCUN VIP VACATIONS,myfngrvnib10ftwrdlfc,2022-04-21 16:10:19,1,19990.00,,0,0,ACTIVE,TPV,CANCELLED


Se definen las variables del extracto de información de transacciones (operaciones) aprobadas o rechazadas de comercios electrónicos que utilizan una pasarela de pagos para procesar las compras de los clientes de su comercio.

# Variables

A continuación se enlista las variables que contiene el archivo:
*Status: Estatus de la transacción al momento de consultar la información transaccional.
*Respuesta: respuesta del motivo de rechazo de la operación proveniente del banco emisor. Si la transacción es completada deberá venir como N/A
*Día: Día de generación de transacción
*Mes: Mes de generación de transacción
*Banco: Emisor del tarjetahabiente que utilizo para procesar la transacción.
*Tipo_Banco: Si es un emisor nacional o internacional.
*Tipo_tarjeta: Si es una tarjeta de crédito, debito prepago u otro tipo.
*Procesamiento: Si el comercio utilizo afiliaciones propias de la pasarela de pago entonces viene con la etiqueta Agregador.
*Comercio: Nombre del comercio en donde se proceso la transacción.
*MerchantID: Identificador del comercio
*Creacion_comercio: Fecha en la que empezo operaciones con la pasarela de pagos.
*TRX: Número de transacciones procedas en ese registro.*Monto: Suma del monto de esas transacciones.
*Con_data_3d: Bandera que permite saber si utilizan la tecnologia de autenticación del usuario llamada 3D secure.
*CC_inicio_trx: Número de Contracargos generados, es decir, reclamación del tarjetahabiente porque no reconoce la operación en dicho comercio.
*CC_Inicio_Monto: Monto que están reclamando como no reconocido.
*Estado_comercio: Bandera para conocer si el comercio aun sigue activo en la pasarela de pagos.
*Origen_trx:
*Detalle_fallida: Clasificación de ciertos motivos de rechazo por fuente que genera el error (existen 3 de importancia: Fallo la autenticación 3DS, entran los sistemas antifraude de la pasarela de pagos, por culpa del tarjetahabiente o por fallo del emisor(banco))

# Descriptiva de las variables

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201629 entries, 0 to 201628
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   STATUS             201629 non-null  object        
 1   RESPUESTA          102893 non-null  object        
 2   DIA                201629 non-null  int64         
 3   MES                201629 non-null  int64         
 4   BANCO              198734 non-null  object        
 5   Tipo_Banco         199590 non-null  object        
 6   TIPO_TARJETA       199691 non-null  object        
 7   PROCESAMIENTO      201629 non-null  object        
 8   COMERCIO           201629 non-null  object        
 9   MERCHANTID         201629 non-null  object        
 10  CREACION_COMERCIO  201629 non-null  datetime64[ns]
 11  TRX                201629 non-null  int64         
 12  MONTO              201629 non-null  float64       
 13  Con_Data_3D        201629 non-null  object  

In [5]:
df.describe()

,DIA,MES,TRX,MONTO,CC_Inicio_TRX,CC_Inicio_Monto
count,201629.000000,201629.000000,201629.000000,2.016290e+05,201629.000000,201629.000000
mean,15.991856,3.001746,10.250807,7.750596e+03,0.029594,19.966612
std,3.185659,0.133461,96.145866,5.472258e+04,0.814738,686.357251
min,11.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,13.000000,3.000000,1.000000,2.690000e+02,0.000000,0.000000
50%,16.000000,3.000000,1.000000,9.606500e+02,0.000000,0.000000
75%,19.000000,3.000000,3.000000,3.375000e+03,0.000000,0.000000
max,21.000000,12.000000,8347.000000,5.959772e+06,130.000000,220077.000000
